In [7]:
print("hello world")
#hello can you see this new change
#please write your name just to make sure we can all edit this

hello world


In [8]:
#Imports
import numpy as np
import scipy as sci

In [9]:
"""
Class for quantum register for n number of qubits
"""
class Quantum_Register:
    def __init__(self,n):
        self.n = n 

In [11]:
newRegister= Quantum_Register(5)
print(newRegister.n)

5
